<a href="https://colab.research.google.com/github/anikatrip/CS2-DS4002/blob/main/3_model_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Read in clean csv data:

In [ ]:
import pandas as pd
# this is the merged, clean dataset that I downloaded onto my computer; the path should be inserted in the code below
vaccination_df = pd.read_csv('file_path')

print("Vaccination DataFrame head:")
display(vaccination_df.head())

## Vaccination Data Preparation:


**Reasoning**:
ensure year is a numeric data type for the linear regression



In [ ]:
vaccination_filtered_df = vaccination_df[['year', 'COVERAGE', 'party']].copy() # Added .copy() to avoid SettingWithCopyWarning
vaccination_filtered_df.loc[:, 'year'] = pd.to_numeric(vaccination_filtered_df['year'])
display(vaccination_filtered_df.head())
print(vaccination_filtered_df['year'].dtype) # look at data type

## Perform Linear Regression on Vaccination Data:


Fit a linear regression model to the vaccination data, including an interaction term between 'year' and 'party' to test for differences in the slopes



In [ ]:
import statsmodels.formula.api as smf

formula = 'COVERAGE ~ year + party + year:party'
vaccination_model = smf.ols(formula, data=vaccination_filtered_df).fit()

## Analyze Results:



Summary of the vaccination model, including coefficients and the p-value of the interaction term (t-test)


In [ ]:
print(vaccination_model.summary())

# Interpret the results
p_value_interaction = vaccination_model.pvalues['year:party[T.Republican]']
print(f"\nP-value for the interaction term (year:party[T.Republican]): {p_value_interaction:.4f}")

# Determine the reference party and the other party (reference is Dem)
reference_party = 'Democrat' # Alphabetically the first party in this dataset
other_party = 'Republican'

# Get coefficients
year_coef = vaccination_model.params['year']
year_party_interaction_coef = vaccination_model.params['year:party[T.Republican]']

print(f"\nCoefficient for 'year' (slope for {reference_party}s): {year_coef:.4f}")
print(f"Coefficient for 'year:party[T.Republican]' (difference in slope for {other_party}s compared to {reference_party}s): {year_party_interaction_coef:.4f}")


##Visualize the Regression:

Create plot showing regression lines for each political party

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Vaccination Data Visualization
plt.figure(figsize=(10, 6))
sns.scatterplot(data=vaccination_filtered_df, x='year', y='COVERAGE', hue='party', alpha=0.6)
# Plot regression line for each party separately
for party in vaccination_filtered_df['party'].unique():
    sns.regplot(data=vaccination_filtered_df[vaccination_filtered_df['party'] == party], x='year', y='COVERAGE', scatter=False, ci=None, color=sns.color_palette()[vaccination_filtered_df['party'].unique().tolist().index(party)])
plt.title('Vaccination Coverage Over Time by Political Party')
plt.xlabel('Year')
plt.ylabel('Coverage (%)')
plt.grid(True)
plt.show()